# Minería De Datos #

## Pregunta 1
* Utilice los siguientes métodos para normalizar el siguiente conjunto de datos<br/>
$$200; 300; 400; 600; 1000$$
   1. Normalización min-max fijando mín = 0 y máx = 1
   2. Normalización Z-score

**Solución 1**
$$v^{'}=\frac{v-min_A}{max_A-min_A}(new \_max_A-new\_min_A)+new\_min_A$$
$max_A=1000$<br/>
$min_A=200$<br/>
$new\_max_A=1$<br/>
$new\_min_A=0$<br/>

In [29]:
import numpy as np
X = np.array([200,300,400,600,1000])
max_A = X.max()
min_A = X.min()
new_max_A = np.array([1])
new_min_A = np.array([0])
new_X = ((X-min_A)/(max_A-min_A))*(new_max_A-new_min_A)+new_min_A
print(new_X)

[0.    0.125 0.25  0.5   1.   ]


**Solución 2**
$$v^{'}=\frac{v-mean_A}{desv_A}$$


In [33]:
import numpy as np
X = np.array([200,300,400,600,1000])
mean_A = X.mean()
desv_A = X.std()
new_X = ((X-mean_A)/desv_A)
print(new_X)

[-1.06066017 -0.70710678 -0.35355339  0.35355339  1.76776695]


## Pregunta 2
* Encuentre la solución de una regresión ridge para un valor genérico $\lambda$ y para el modelo lineal <br/><br/>
$$y=\beta_0+\beta_1 X +\epsilon$$<br/>
Muestre que para $\lambda=0.4$ la solución de la regresión ridge es $\hat{y}=40+1.75X$.<br/><br/>
Datos $X^{T}=(-2,-1,-1,-1, 0, 1, 2, 2)^{T}$; $y^{T}=(35, 40, 36, 38, 40, 43, 45, 43)^{T}$

**Solución**
$$\hat{\beta}^{ridge} = (X^T X + \lambda I)^{-1}X^Ty$$ 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
X = np.array([-2,-1,-1,-1, 0, 1, 2, 2])
y = np.array([35, 40, 36, 38, 40, 43, 45, 43])
plt.scatter(x, y, alpha=0.5)

In [99]:
import numpy as np
x1 = np.array([-2,-1,-1,-1, 0, 1, 2, 2])
#x1 = (x1-x1.mean())
#/x1.std()
X = np.matrix(x1).transpose()
y = np.matrix([35, 40, 36, 38, 40, 43, 45, 43]).transpose()

from numpy.linalg import inv
beta_ridge_0 = y.mean()
beta_ridge = inv(X.transpose()*X + 0.4*np.identity(1))*(X.transpose()*y)
print(beta_ridge)

[[2.13414634]]


In [80]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=0.4,fit_intercept=False)
a=clf.fit(X, y) 
print(a.coef_)

[[38.0952381   2.13414634]]


## Pregunta 3
* Demuestre que la regresión RIDGE es equivalente al siguiente problema de optimización
$$\hat{\beta}^c = \arg min{\sum_{i=1}^N [y_i - \beta_0^c - \sum_{j=1}^p(x_{i,j}-\bar{x}_j)\beta_j^c]^2+\lambda \sum_{j=1}^p(\beta_j^c)^2}$$
Encuentre la solución $\hat{\beta}^c$ y comparela con la solución original de la regresión ridge.

## Pregunta 5
* Implemente el método del gradiente proximal para el problema LASSO. Utilice esta implementación para encontrar los dos factores físico-quimicos más importantes para la calidad del vino portugues [Vinho Verde](http://archive.ics.uci.edu/ml/datasets/Wine+Quality).<br/>
Nota: Para el criterio de parada utilice el número de iteraciones (e.g. 1000).

### Algortimo LASSO

1. $\beta$ aleatorio
2. $\tau$>0
3. $\beta^{i+1} = \displaystyle {prox_{\tau \delta f}^{}} \left( \beta^i +2\tau X^T(y-X\beta^i) \right) $

**Solución**

Dado $f(\beta) = \lambda ||\beta||_1$, tenemos que : $$prox_{\tau \delta f}(\beta)$$ puede ser expresado como:
$$max(0,\beta-\lambda \tau)-max(0,-(\beta+\lambda \tau));$$
Ahora tomamos $\beta=\beta^i +2\tau X^T(y-X\beta^i)$ y realizamos 1000 iteraciones del método.

### Función Proximal

In [459]:
def prox_l1(beta, plambda):
    vect = np.array(beta)-plambda
    vect2 = -np.array(beta)-plambda
    a = np.zeros((n,1))
    z = np.max(np.concatenate((a, vect),axis=1), axis=1) - np.max(np.concatenate((a, vect2),axis=1), axis=1)
    return np.matrix(z).transpose()

### Función de Costo

In [460]:
def lasso_cost(y,X,beta,plambda):
    vect = np.array(y)-np.array(X*beta)
    return np.linalg.norm(y-X*beta, ord=2)**2 + plambda * np.linalg.norm(beta, ord=1)

### Parámetros de Entrada

In [461]:
import pandas as pd
import os
os.chdir("C:\Python")
os.getcwd()

'C:\\Python'

In [462]:
# Los datos de entrada deben estar en el direcotrio local

df = pd.read_csv('winequality-red.csv',sep=";")
X_train = np.matrix(df.iloc[:,0:11])
y_train = np.matrix(df.iloc[:,11:12])
y_train.shape
#x_test = Array(d["x_test"]);
#y_test = Array(d["y_test"]);

(1599, 1)

In [469]:
#parámetros

beta = np.matrix(np.zeros(11)).transpose()
tau = 0.0002
plambda = 0.01
gamma = 0.5
maxiter = 10000
tol = 1e-3
res = 1
it = 0
costos = np.array([0])
beta_prev = beta

In [472]:
for i in range(maxiter):
    it =it+1
    grad_g = np.array(-2*X_train.transpose()*(y_train-X_train*beta))
    z = prox_l1(beta + tau*grad_g , tau*plambda)
    cost = lasso_cost(y_train,X_train,z,plambda)
    beta_prev = beta
    beta = z
    #costos = np.concatenate((costos, cost),axis=0)

C:\Users\eduardo\Anaconda3\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
